# RareSkills Zero Knowledge Week 1


## Modular arithmetic essentials
### Benchmarks
Here is how you do modular arithmetic in python. Run the two cells below and note how different their execution times are. You should use `pow` instead of doing modular arithmetic with the elementary operators.

In [11]:
import time

s = time.time()
ans = 1002583 ** 939001 % 2003951
t = time.time()

print("calculating {} took {:.2f} seconds".format(ans, t - s))

calculating 1590868 took 3.25 seconds


In [2]:
s = time.time()
ans = pow(1002583,939001,2003951)
t = time.time()

print("calculating {} operation took {:.2f} seconds".format(ans, t - s))

calculating 1590868 operation took 0.00 seconds


### Encrypting a number

If we start with a generator `g` and raise it to a power `s` mod `n`, then in general we cannot determine `s` even if `g` and `n` are known. This is the discrete logarithm problem.

We can say we "encrypt" `s` using the scheme

$$ E(s) = g^{s} \pmod n $$

But this means we need to choose `g` and `n` carefully. If `g` is 3, `s` is 2, and `n` is a large number, we can intelligently guess what `s` is in this case. We want to pick `g` and `n` such that all members "wrap around." We won't concern ourselves with this here, but you should be aware of it. The other issue is not picking a sufficiently large `n`. 

### Computing the discrete logarithm
If we do `s_p = g^s mod n`, and make `g` and `n` public, then it is infeasible to compute `s`. But this is only true if `n` is sufficiently large.

In the following example, we provide `s_p`, `g`, and `n`. Use python to bruteforce the solution to find `s`.

$$g^{ac} = (g^{a})^{c} \pmod p$$

In [8]:
n = 9551
g = 5
encrypted_number = 5666

<font color='red'>**Assignment 1: Use python to bruteforce the solution to find `s`**</font>

In [9]:
# Your Solution and Answer Here
student_solution = 0
for s in range(n):
    if pow(g, s, n) == encrypted_number:
        student_solution = s
        break
print ('Decrypted number', student_solution, 'check', encrypted_number, '==', pow(g, student_solution, n))

Decrypted number 2531 check 5666 == 5666


In [10]:
assert pow(g, student_solution, n) == encrypted_number
print("student_solution is {}".format(student_solution))

student_solution is 2531


### Computing the discrete logarithm the smart way
The smart way to do optimal algorithms is to [copy and paste from stackoverflow](https://stackoverflow.com/a/58665206/2079806). You do not have to understand how the library works

<font color='red'>**Assignment 2: Break the cryptography below and obtain the secret number**</font>

In [11]:
n = 1000004119
g = 5
encrypted_number = 767805982

In [12]:
# Your Solution and Answer Here
from sympy.ntheory import discrete_log

student_solution = discrete_log(n, encrypted_number, g)

In [13]:
assert pow(g, student_solution, n) == encrypted_number
print("student_solution is {}".format(student_solution))

student_solution is 420


## How Zero Knowledge Addition Works

A Zero Knowledge Proof verifies a computation is true without revealing a the inputs to the computation.

That is, if a + b = c, then we want a "trapdoor function" E(x) such that E(a)■E(b) = E(c) where ■ is the binary operator of the group E(x) is an element of.

So to prove a + b = c without revealing a, b, or c, we apply A = E(a), B = E(b), C = E(c) and give (A, B, C) to a verifier.

The verifier will then check A ■ B == C.

Although they don't know the values A, B, and C "came from", they do know the unencrypted values form a valid sum.

## Zero Knowledge Addition
The following property is very important. It lets us verify the addition of numbers (integers under addition) using another group: $g^x \pmod p$ under multiplication. Because of the discrete logarithm, we cannot easily go from $g^x \pmod p$ to recover x.

a + b = c is homomorphic to $g^ag^b = g^c \pmod p$.

$$ g^{a}g^{b} = g^{a + b} \pmod n $$

Try a few different values of `a` and `b` yourself to see it in action

In [20]:
import random
for i in range(10):
    g = random.randint(0, n)
    a = random.randint(0, n)
    b = random.randint(0, n)
    print(pow(g, a, n) * pow(g, b, n) == pow(g, a + b, n))

False
False
False
False
False
False
False
False
False
False


Wait what?! That's supposed to be true?

<font color='red'>**Assignment 3: The code above has a bug. What is the bug?**</font>

In [21]:
# gotta do MOD N after multiplication!
import random
for i in range(10):
    g = random.randint(0, n)
    a = random.randint(0, n)
    b = random.randint(0, n)
    print(pow(g, a, n) * pow(g, b, n) % n == pow(g, a + b, n))

True
True
True
True
True
True
True
True
True
True


## Zero Knowledge Subtraction
We can also encrypt the operation a - b. This is the same as 

$$ g^{a}g^{-b} = g^{a-b} $$

But we can't just stick a negative sign in front of the exponent

In [22]:
g = 57
n = 101
g ** -5 % n

1.6619797259514097e-09

The above is not an integer! However, python is smart enough to know what you are doing if you use `pow`. To test this, we expect

$$ g^{-a}g^{a} = 1 $$

because g^0 is 1, and `a - a = 0`.

In [23]:
a = 22
pow(g, a, n) * pow(g, -a, n) % n == 1

True

So what magic is happening behind the scenes? The formula below is used to compute the modular inverse.

$$ a^{-1} = a^{n - 2} \pmod n $$

In [24]:
a_inv = a ** (n - 2) % n
a_inv * a % n == 1

True

## Multiplication by a constant
Multiplication by a constant is really just repeated addition

$$ (g^{a})^{4} = g^{a} g^{a} g^{a} g^{a} $$

This allows us to "multiply by a constant" in a zero knowledge fashion.

In [25]:
a = 15
pow(g, a, n) * pow(g, a, n) * pow(g, a, n) * pow(g, a, n) % n == pow(g, a * 4, n)

True

Of course, it would be annoying to multiply like that if you have a big coefficient, so the following is better

In [26]:
pow(pow(g, a, n), 4 , n) == pow(g, a * 4, n)

True

## I know the solution to a systems of equations

Now you should be able to prove to me that you know the solution to in a zero knowledge fashion

$$
2x + 8y = 7944\\
5x + 3y = 4764
$$

<font color='red'>**Assignment 4: conver the above equation to a zk proof system where you can prove to a verifier (with an agreed upon g and n) that you know the solution to an agreed upon system of equation, without revealing the solution**</font>

In [12]:
import sympy

p = 10007
print('is', p, 'prime?', sympy.isprime(p))

g = sympy.primitive_root(p)
print('generator is', g)
assert g == 5 # i'm not sure if primitive_root() is deterministic

gx = 8134 # g^x, x is secret
gy = 9325 # g^y, y is secret

# we want: gx^2 * gy^8 == g^7944
assert (pow(gx, 2, p) * pow(gy, 8, p) - pow(g, 7944, p)) % p == 0
# we want: gx^5 * gy^3 == g^4764
assert (pow(gx, 5, p) * pow(gy, 3, p) - pow(g, 4764, p)) % p == 0

# below is not really homework, just playing around
print('x', sympy.ntheory.discrete_log(p, gx, g))
print('y', sympy.ntheory.discrete_log(p, gy, g))

is 10007 prime? True
generator is 5
x 420
y 888


In [25]:
p = sympy.nextprime(10**20)
print('is', p, 'prime?', sympy.isprime(p))

g = sympy.primitive_root(p)
print('generator is', g)

gx = pow(g, 420, p)
gy = pow(g, 888, p)
print('gx', gx, 'gy', gy)

# we want: gx^2 * gy^8 == g^7944
assert (pow(gx, 2, p) * pow(gy, 8, p) - pow(g, 7944, p)) % p == 0
# we want: gx^5 * gy^3 == g^4764
assert (pow(gx, 5, p) * pow(gy, 3, p) - pow(g, 4764, p)) % p == 0

# below is not really homework, just playing around
s = time.time()
print('x', sympy.ntheory.discrete_log(p, gx, g))
print('y', sympy.ntheory.discrete_log(p, gy, g))
t = time.time()
print("calculating two discrete logs took {:.2f} seconds".format(t - s))

is 100000000000000000039 prime? True
generator is 3
gx 75691350556124262060 gy 88489376898100166924
x 420
y 888
calculating two discrete logs took 33.79 seconds


## Convert a rational number to a finite field element

<font color='red'>**Assignment 5: Compute 53/192 + 61/511 (mod 1033) in python. Show your work. You are not required to implement euclid's algorithm. Show it is equal to the original rational number sum: https://www.wolframalpha.com/input?i=53%2F192+%2B+61%2F511**</font>

In [28]:
N = 1033

sum = (53 * pow(192, -1, N) + 61 * pow(511, -1, N)) % N
print(sum)
# sum == 38795 / 98112
print((sum * 98112 - 38795) % N)

514
0
